## Capstone project Week3.

In [1]:
import pandas as pd

### 1. Data set creation and manipulation

In [113]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.DataFrame(data=table[0])
df

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,Rouge
2,M1B,Scarborough,Malvern
3,M1C,Scarborough,Highland Creek
4,M1C,Scarborough,Rouge Hill
...,...,...,...
282,M9V,Etobicoke,Thistletown
283,M9W,Etobicoke,Northwest
284,M9X,Not assigned,Not assigned
285,M9Y,Not assigned,Not assigned


In [124]:
tdf = torontoNB[torontoNB.Borough !='Not assigned']
tdf
#na = torontoNB[ torontoNB['Borough'] == 'Not assigned' ].index
#torontoNB.drop(na, inplace=True)
#torontoNB

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,Rouge
2,M1B,Scarborough,Malvern
3,M1C,Scarborough,Highland Creek
4,M1C,Scarborough,Rouge Hill
5,M1C,Scarborough,Port Union
...,...,...,...
279,M9V,Etobicoke,Mount Olive
280,M9V,Etobicoke,Silverstone
281,M9V,Etobicoke,South Steeles
282,M9V,Etobicoke,Thistletown


In [122]:
dftt = tdf(['']

In [126]:
ttdf = tdf.groupby('Postcode')['Neighborhood'].apply(', '.join).reset_index
ttdf()

,Postcode,Neighborhood
0,M1B,"Rouge , Malvern"
1,M1C,"Highland Creek , Rouge Hill , Port Union"
2,M1E,"Guildwood , Morningside , West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
...,...,...
98,M9N,Weston
99,M9P,Westmount
100,M9R,"Kingsview Village , Martin Grove Gardens , Ric..."
101,M9V,"Albion Gardens , Beaumond Heights , Humbergate..."


In [133]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.DataFrame(data=table[0])
df

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Cleaning the original dataframe

In [134]:
# Dropping not assigneed values from Borough
df = df.drop(df[(df.Borough == "Not assigned")].index)
# Replace Not assigned, and NaN values, of Neighborhoods with Borough value 
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)
df.Neighborhood.fillna(df.Borough, inplace=True)
# Drop duplicate rows
df=df.drop_duplicates()


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


In [135]:
df.update(df.Neighborhood.loc[lambda x: x.str.contains('title')].str.extract('title=\"([^\"]*)',expand=False))
df.update(df.Borough.loc[ lambda x: x.str.contains('title')].str.extract('title=\"([^\"]*)',expand=False))
df.update(df.Neighborhood.loc[lambda x: x.str.contains('Toronto')].str.replace(", Toronto",""))
df.update(df.Neighborhood.loc[lambda x: x.str.contains('Toronto')].str.replace("\(Toronto\)",""))

Creating the final dataframe that includes unique values for the Postcode column

In [136]:
ndf = pd.DataFrame({'Postcode':df.Postcode.unique()})
ndf['Borough'] = pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in ndf.iterrows())
ndf['Neighborhood'] = pd.Series(list(set(df['Neighborhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in ndf.iterrows())
ndf['Neighborhood'] = ndf['Neighborhood'].apply(lambda x: ', '.join(x))
ndf.dtypes

Postcode        object
Borough         object
Neighborhood    object
dtype: object

In [138]:
ndf.shape

(103, 3)

In [140]:
ndf.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### 2. Creating the Toronto dataframe with latitude and longitude

In [143]:
ll= pd.read_csv("http://cocl.us/Geospatial_data")
ll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
ll.set_index("Postcode")
ndf.set_index("Postcode")
torontodf=pd.merge(ndf, ll)
torontodf.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### 3. Visualization

In [144]:
import folium
import numpy as np 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [145]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, are 43.653963, -79.387207.


In [148]:
torontomap = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(torontodf['Latitude'], torontodf['Longitude'], torontodf['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(torontomap)  
    
torontomap

In [155]:
CLIENT_ID = 'TDQMJK3XNUQI0OZOWSBZCED02JBCALMPK0DY1ZRXHAW2LPXR' # your Foursquare ID
CLIENT_SECRET = 'E1AZT3BYD0MDKS5GP1PHAWCMATNPD2BD1CJEOHUCNWZCEUV2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

#### Explore venues in Toronto

In [156]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [157]:
toronto_venues = getNearbyVenues(names=torontodf['Neighborhood'],
                                   latitudes=torontodf['Latitude'],
                                   longitudes=torontodf['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park
Queen's Park
Malvern, Rouge
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Princess Gardens, West Deane Park, Islington, Martin Grove, Cloverdale
Port Union, Rouge Hill, Highland Creek
Don Mills South, Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Markland Wood, Bloordale Gardens, Eringate, Old Burnhamthorpe
Morningside, Guildwood, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Adelaide, King, Richmond
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Oriole, Henry Farm
York University, Northwood Park
East Toronto
Toronto Islands, Harbourfront East, Union Station
Little Portugal, Trinity
East Birchmount Park, Kennedy Park, Ionview
Bayview Village
CFB Toronto, Downsview East
Riverdale, The Danfort

KeyError: 'groups'

In [158]:
print(toronto_venues.shape)
toronto_venues.head()

NameError: name 'toronto_venues' is not defined